In [ ]:

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
     

# NOTE: This Colab is only for documentation

## Constrained Trajectory Optimization

Trajectory optimization forms the backbone of model-based optimal control with
myriad applications in robot mobility and manipulation. The problem formulation
adopted within this library is as follows: consider a system with state $x \in
\mathbb{R}^n$, control input $u \in \mathbb{R}^m$, subject to the
discrete-time dynamics: 
\begin{equation} x_{k+1} = f_k(x_k, u_k), \quad k \in
\mathbb{N}_{\geq 0}.
\end{equation}
Let $T \in \mathbb{N}_{>0}$ be some fixed
planning horizon. Given some initial state $x_0$, the trajectory optimization
problem is as follows:
$$
\begin{eqnarray} \min_{\bf{u}, \bf{x}} \qquad
&\sum_{k=0}^{T-1} l_k(x_k, u_k) + l_T(x_N)  \\ 
\textrm{s.t.}
\qquad & k=0,\ldots,T-1: \quad \begin{cases} x_{k+1} = f(x_k, u_k) \\ 
c_k^u(u_k)
\geq 0 \\ c_{k+1}^x(x_{k+1}) \geq 0 \end{cases},
\end{eqnarray}
$$
where we use
$(\bf{u}, \bf{x})$ to denote the concatenations $(u_0, \ldots, u_{T-1})$ and
$(x_0, \ldots, x_{T})$, respectively. Here, 
$$
\begin{eqnarray} &l_k :
&\mathbb{R}^n \times \mathbb{R}^m \rightarrow \mathbb{R}_{\geq 0} \qquad
&\textrm{running cost} \\ 
&l_T: &\mathbb{R}^n \rightarrow \mathbb{R}_{\geq 0}
\qquad & \textrm{terminal cost} \\ 
&c_k^x : &\mathbb{R}^n \rightarrow
\mathbb{R}^{n_x} \qquad &\textrm{state-constraint function} \\ 
&c_k^u:
&\mathbb{R}^m \rightarrow \mathbb{R}^{n_u} \qquad &\textrm{control-constraint
function} 
\end{eqnarray}
$$
We assume that the control constraint encodes simple
box constraints: $\underline{u} \leq u_k \leq \overline{u}$.

This library allows specifying problems of the above form and provides a
Shooting-based SQP solver. For algorithmic details, please see
[Singh et al.](https://arxiv.org/pdf/2109.07081.pdf)

## Specifying Problems

We demonstrate how to define and solve a problem using the `shootsqp` library.

### Basic Setup

Given the problem formulation above, user must define the following*:

| Input                              | Description                             |
| ---------------------------------- | --------------------------------------- |
| `n: int`                           | state-dimension                        |
| `m: int`                           | control-dimension                      |
| `T: int`                           | horizon                                 |
| `dynamics: Callable[[x, u, k], x]` | modeling the discrete-time dynamics function $f_k$.                    |
| `cost: Callable[[x, u, k,  *cost_params], float]`  | modeling the stage-$k$ cost function $l_k$, where `cost_params` is a tuple of custom parameters.<br> Note that this function must represent both the stage-wise ($\{l_k\}_{k=0}^{T-1}$)  and terminal ($l_T$) costs, <br> e.g., using `jnp.where`.    <br> |
| `control_bounds: Tuple[u, u]`      | Tuple of lower ($\underline{u}$) and  upper ($\overline{u}$) control bound  vectors. |
| `state_constraint: Callable[[x, k, *constraint_params], ndarray]` | modeling the stage-$k$ state-constraint function $c_k^x$ where `constraint_params` <br> is a tuple of custom parameters. |


---


*Type notation `x: (n,)-ndarray, u: (m,)-ndarray, k: int`.

If there are no further constraints, we can define a dictionary of solver
options (see below for a description of some key options) and instantiate a
`ShootSQP` solver instance as follows:

```python
prob = shootsqp.ShootSQP(n, m, T, dynamics, cost, control_bounds, state_constraint, **solver_options)
```

The returned object's `solve` method can now be used to solve multiple problems
(without re-instantiation), given:

*   an initial state `x_0: (n,)-ndarray`
*   control trajectory guess `U0: (T, m)-ndarray`
*   (optional) state trajectory guess `X0: (T+1, n)-ndarray`
*   problem parameters `params: Tuple[cost_params_tuple,
    constraint_params_tuple]`, corresponding to the custom parameter tuples for
    the cost and state-constraint functions.

If however, there are components of the state that lie within the
$\mathcal{S}^1$ sphere (as is often the case for Robotics), please see the $\mathcal{S}^1$ manifold section below for additional details regarding solver
instantiation.

### Some Key Solver Options

For a list of all solver options, please see the paper reference and
*shootsqp.py*. All options are accessible via `prob.opt`. NOTE: once the `solve`
method is called with some problem data, not all options can be reset due to jit
pre-compilation. Future releases will make clear what options are mutable and
what are held fixed. Some important solver options are described below:

| Option            | Description                                              |
| ----------------- | -------------------------------------------------------- |
| `method: str`     | SQP variation. One of {`SQP_METHODS.OPEN`, `SQP_METHODS.STABLE`, `SQP_METHODS.SENS`, `SQP_METHODS.APPROX_SENS`}. <br> `OPEN` variant uses open-loop rollouts; `STABLE` variant features closed-loop rollouts in the update, using TV-LQR generated <br> gains. `SENS` also features closed-loop rollouts, but using the "DDP-sensitivity" method from the cited reference. <br>`APPROX_SENS` uses an approximation of the sensitivity computation.
| `proj_init: bool` | If `True`, user must also supply an initial guess for the state trajectory `X0 : (T+1, n)-ndarray`. The solver will begin <br> optimization by performing a closed-loop  rollout to generate a dynamically feasible initial trajectory that is close to `X0`. |
| `hess: str`       | Hessian type. One of {`full`, `gn`}. `full` refers to the full Lagrangian Hessian, while `gn` refers to the Gauss-Newton <br> approximation (which drops 2nd order dynamics gradients).                 |
| `qp_solver: str`  | Solver type for QP sub-problem. One of {`QP_SOLVERS.QP_ECOS, QP_SOLVERS.QP_CVX, QP_SOLVERS.QP_ALILQR`}. <br> The first two correspond to cvxpy solvers. The third uses trajax's constrained-iLQR method. |
| `do_log: bool`    | Enables/disables solution progress logging. Set to  `False` for speed-sensitive applications.  |
| `max_iter: int`   | Maximum SQP iterations. Note that the first time the `solve` method is called, it needs to jit all relevant methods.<br> So, for speed-sensitive applications, it is recommended to initialize the problem with `max_iter = 1`, solve a dummy <br> instance to jit everything, and then  reset this option as: `prob.opt.max_iter =   <desired_value>`.        |

---

## Handling $\mathcal{S}^1$ manifold constraints

For systems where components of the state lie on the $\mathcal{S}^1$ sphere
(or a subset thereof), one must be careful with the handling of these states
during optimization. As a concrete example, suppose $\theta$ is a component of
the state that is supposed to lie within $\mathcal{S}^1$. There are two
distinct scenarios for the treatment of $\theta$:

*   Case 1: Allowable range: $\theta \in [-\pi, \pi]$. One can treat
    $\theta$ as an "unbounded" variable (i.e., not subject to any limit
    constraints). This is because $\theta$ is likely to appear within
    objective and constraints only via trignometric transformations.
*   Case 2: Allowable range: $\theta \in [\underline{\theta},
    \overline{\theta}]$. In this case, one should treat $\theta$ as just
    another variable on the real-line, subject to known bounds.

Operationally, we only need to worry about Case 1 since in order for the
closed-loop SQP variants to correctly apply feedback on the "error" in
$\theta$ during rollouts, the rollout must correctly compute differences on
the $\mathcal{S}^1$ sphere. To do so, we append to the ShootSQP object
initialization a tuple of indices indicating the components of state allowed to
lie on the full $\mathcal{S}^$ sphere. Internally, the solver handles any
necessary angle-wrapping between optimization iterations.

For components of state corresponding to Case 2 above, we incorporate the bound
constraints as usual within the `state_constraint` function.